# The Pytorch Implementation of [Denoising Diffusion Probabilistic Model](https://arxiv.org/pdf/2006.11239.pdf)

### 아래의 Shell을 통해 유관 패키지 전체를 다운로드할 수 있습니다.

#### python3.8 기반으로 본 코드를 테스트 해볼 것을 추천드립니다.

In [ ]:
'''CUDA 11.1 기반일 경우'''
!python -m pip install -r ../requirements_for_cuda111.txt
'''CUDA 10.1 기반일 경우'''
#!python -m pip install -r ../requirements_for_cuda101.txt

'''CUDA 11.1 기반에서 torch만 설치하고 싶을 경우'''
#!pip install torch==1.9.0+cu101 torchvision==0.10.0+cu101 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
'''CUDA 11.1 기반에서 torch만 설치하고 싶을 경우'''
#!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html


In [ ]:
import os
import json
import argparse
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

from lib.model import UNet
import lib.dataset as dataset
from lib.diffusion import GaussianDiffusion, make_beta_schedule
from lib.util import accumulate, samples_fn, progressive_samples_fn, bpd_fn, validate

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [ ]:
print(torch.__version__) # 1.9.0+cu1x1 ??
print(pl.__version__) # 0.8.5 ??

# Denoising Diffusion Probabilistic Model

* 본 모델은 diffusion probabilistic model의 일종으로, diffusion model의 특정 parameterization 방식이 denoising score matching의 방법론과 동일해질 수 있음을 보인 논문입니다. 기서 diffusion probabilistic model은 markov chain의 방식을 활용한 모델의 일종으로 이해해주시면 됩니다.

* Multiple transition 과정에서 점진적으로 data에 noise를 inject할 수 있는 markov chain 구조를 통해, 본 저자는 diffusion model이 denoising score matching과 동일해질 수 있다고 이야기하고 있습니다.

* diffusion model에 대한 background에 대해서는, 본 논문의 background section을 참고해주시면 감사드리겠습니다. 본 notebook에서는 구현 단위의 방법론에 대해서 주요하게 다루도록 하겠습니다.

## Forward and reverse process


* 아래의 그림과 같이, diffusion model은 forward process와 reverse process 각각으로 구성되어 있으며, 각각의 transition은 markov chain의 방식으로 이루어집니다. 
* forward process의 경우 fixed markov chain을 활용하며, 각 단계별로 지정된 variance schedule에 따라 gaussian noise를 부여하는 식으로 transition이 이루어집니다. 허나, reverse process의 경우 각각의 transition 별로 정의된 gaussian transition에 대해 learnable parameter를 부여한다는 점에서 forward process와 다른 점이 있습니다.

<img src="ddpm_image/image1.PNG" width="500">

# Diffusion models and denoising autoencoders

* 위에서 언급했다시피, 본 저자가 주목한 부분은 diffusion model을 마치 denoising autoencoder 처럼 해석해볼 수 없겠냐는 것입니다. diffusion model에 대한 적절한 variational bound를 유도해낼 수 있다면 둘의 작동원리는 매우 유사해지기 때문이죠. denoising autoencoder 또한 noise가 존재하는 데이터에 대해 noise를 없애주는, denoising 방법이 주요한 원리이며, 이는 diffusion model의 reverse process와 매우 유사하다는 것을 알 수 있습니다.
* 고정된 forward process에 대하여, reverse process에 활용되는 parameter를 학습하는 방법은 아래와 같습니다.

<img src="ddpm_image/image2.PNG" width="500">

* 자세한 notation은 논문을 참고해주시기 바라며, 요약하자면 총 T번의 transition이 이루어지는 상황에서 reverse process의 t 시점에 해당하는 data sample에 대한 mean parameter는 forward process에서 얻어낸 posterior mean값과 가깝도록 학습해주면 된다는 뜻입니다. 이를 조금 더 정리해서 나타내면 해당 mean에 대한 parameterization을 아래와 같이 표현할 수 있습니다.

<img src="ddpm_image/image4.PNG" width="800">

* 해당 식에 대해 reparameterization을 활용하면, 결국 해당 과정에서 최종적으로 approximate해야하는 대상은 입실론 쎄타 값으로 변화하게 되며, 아래와 같은 loss objective가 만들어집니다. 
    
<img src="ddpm_image/image5.PNG" width="500">

* 해당 식은 다양한 noisy scale에 대한 denoising score matching 방식과 매우 유사하다는 결론을 낼 수 있습니다.

* 본 논문에선 self-attention을 포함한 U-Net structure를 backbone 모델로 활용하고 있습니다. 

In [ ]:
class DDP(pl.LightningModule):
    def __init__(self, conf):
        super().__init__()

        self.conf  = conf
        self.save_hyperparameters()

        self.model = UNet(self.conf.model.in_channel,
                          self.conf.model.channel,
                          channel_multiplier=self.conf.model.channel_multiplier,
                          n_res_blocks=self.conf.model.n_res_blocks,
                          attn_strides=self.conf.model.attn_strides,
                          dropout=self.conf.model.dropout,
                          fold=self.conf.model.fold,
                          )

        self.ema   = UNet(self.conf.model.in_channel,
                          self.conf.model.channel,
                          channel_multiplier=self.conf.model.channel_multiplier,
                          n_res_blocks=self.conf.model.n_res_blocks,
                          attn_strides=self.conf.model.attn_strides,
                          dropout=self.conf.model.dropout,
                          fold=self.conf.model.fold,
                          )

        self.betas = make_beta_schedule(schedule=self.conf.model.schedule.type,
                                        start=self.conf.model.schedule.beta_start,
                                        end=self.conf.model.schedule.beta_end,
                                        n_timestep=self.conf.model.schedule.n_timestep)

        self.diffusion = GaussianDiffusion(betas=self.betas,
                                           model_mean_type=self.conf.model.mean_type,
                                           model_var_type=self.conf.model.var_type,
                                           loss_type=self.conf.model.loss_type)



    def setup(self, stage):

        self.train_set, self.valid_set = dataset.get_train_data(self.conf)

    def forward(self, x):

        return self.diffusion.p_sample_loop(self.model, x.shape)

    def configure_optimizers(self):

        if self.conf.training.optimizer.type == 'adam':
            optimizer = optim.Adam(self.model.parameters(), lr=self.conf.training.optimizer.lr)
        else:
            raise NotImplementedError

        return optimizer

    def training_step(self, batch, batch_nb):

        img, _ = batch
        time   = (torch.rand(img.shape[0]) * 1000).type(torch.int64).to(img.device)
        loss   = self.diffusion.training_losses(self.model, img, time).mean()

        accumulate(self.ema, self.model.module if isinstance(self.model, nn.DataParallel) else self.model, 0.9999)

        tensorboard_logs = {'train_loss': loss}

        return {'loss': loss, 'log': tensorboard_logs}

    def train_dataloader(self):

        train_loader = DataLoader(self.train_set,
                                  batch_size=self.conf.training.dataloader.batch_size,
                                  shuffle=True,
                                  num_workers=self.conf.training.dataloader.num_workers,
                                  pin_memory=True,
                                  drop_last=self.conf.training.dataloader.drop_last)

        return train_loader

    def validation_step(self, batch, batch_nb):

        img, _ = batch
        time   = (torch.rand(img.shape[0]) * 1000).type(torch.int64).to(img.device)
        loss   = self.diffusion.training_losses(self.ema, img, time).mean()

        return {'val_loss': loss}

    def validation_epoch_end(self, outputs):

        avg_loss         = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}

        shape  = (16, 3, self.conf.dataset.resolution, self.conf.dataset.resolution)
        sample = progressive_samples_fn(self.ema, self.diffusion, shape, device='cuda' if self.on_gpu else 'cpu')

        grid = make_grid(sample['samples'], nrow=4)
        self.logger.experiment.add_image(f'generated_images', grid, self.current_epoch)

        grid = make_grid(sample['progressive_samples'].reshape(-1, 3, self.conf.dataset.resolution, self.conf.dataset.resolution), nrow=20)
        self.logger.experiment.add_image(f'progressive_generated_images', grid, self.current_epoch)
        
        return {'val_loss': avg_loss, 'log': tensorboard_logs}

    def val_dataloader(self):
        valid_loader = DataLoader(self.valid_set,
                                  batch_size=self.conf.validation.dataloader.batch_size,
                                  shuffle=False,
                                  num_workers=self.conf.validation.dataloader.num_workers,
                                  pin_memory=True,
                                  drop_last=self.conf.validation.dataloader.drop_last)

        return valid_loader

# Training

* 본 모델의 training을 위한 hyperparameter 및 각 argument에 대한 선언은 아래와 같이 이루어집니다.
* cifar10 데이터에 대한 각 hyperparameter의 경우 해당 config 파일을 자세히 참고해주시길 부탁드립니다.

In [ ]:
class obj(object):
    def __init__(self, d):
        for a, b in d.items():
            if isinstance(b, (list, tuple)):
               setattr(self, a, [obj(x) if isinstance(x, dict) else x for x in b])
            else:
               setattr(self, a, obj(b) if isinstance(b, dict) else b)

import easydict

args = easydict.EasyDict({
        "train": False,
        "config": 'config/diffusion_cifar10.json',
        "ckpt_dir": 'ckpts',
        "ckpt_freq": 5,
        "n_gpu": 1,
        "model_dir": 'ckpts/last.ckpt',
        "sample_dir": 'samples',
        "prog_sample_freq": 200,
        "n_samples": 100
    })

    
path_to_config = args.config
with open(path_to_config, 'r') as f:
    conf = json.load(f)

conf = obj(conf)
denoising_diffusion_model = DDP(conf)

In [ ]:
if pl.__version__ == '0.8.5':
    checkpoint_callback = ModelCheckpoint(filepath=os.path.join(args.ckpt_dir, 'ddp_{epoch:02d}-{val_loss:.2f}'),
                                      monitor='val_loss',
                                      verbose=False,
                                      save_last=True,
                                      save_top_k=-1,
                                      save_weights_only=True,
                                      mode='auto',
                                      period=args.ckpt_freq,
                                      prefix='')
    
    
else:
    checkpoint_callback = ModelCheckpoint(dirpath=os.path.join(args.ckpt_dir, 'ddp_{epoch:02d}-{val_loss:.2f}'),
                                      monitor='val_loss',
                                      verbose=False,
                                      save_last=True,
                                      save_top_k=-1,
                                      save_weights_only=True,
                                      mode='auto',
                                      period=args.ckpt_freq)
#     except:
#         print('The present library version of pytorch lightning is ',pl.__version__)
#         print('Please check if library requirements are satisfied')
    


In [ ]:


trainer = pl.Trainer(fast_dev_run=False,
                     gpus=args.n_gpu,
                     max_steps=conf.training.n_iter,
                     precision=conf.model.precision,
                     gradient_clip_val=1.,
                     progress_bar_refresh_rate=1,
                     checkpoint_callback=checkpoint_callback,
                     check_val_every_n_epoch=10000)

trainer.fit(denoising_diffusion_model)

# Testing and Sampling

## 결과 해석

* Training된 모델에 대하여, 1) 임의의 random sample에 대한 generation 능력 테스트 및 2) noise에서 각 sample이 progressively 잘 생성되는지 확인하기 위해 아래와 같이 Image generation을 진행하였습니다.
* 해당 실험 결과, CIFAR-10에서 볼법한 이미지들이 잘 생성되는 것을 확인할 수 있습니다. (이미지가 본 노트북에 보이지 않는 경우가 있습니다. 이를 방지하기 위해 저장된 이미지를 추가로 활용하시길 바랍니다.)

In [ ]:
denoising_diffusion_model.cuda()
state_dict = torch.load('ckpts/last.ckpt')
denoising_diffusion_model.load_state_dict(state_dict['state_dict'])
denoising_diffusion_model.eval()

sample = progressive_samples_fn(denoising_diffusion_model.ema,
                                denoising_diffusion_model.diffusion,
                                (100, 3, conf.dataset.resolution, conf.dataset.resolution),
                                device='cuda',
                                include_x0_pred_freq=args.prog_sample_freq)

if not os.path.exists(args.sample_dir):
    os.mkdir(args.sample_dir)

plt.figure(figsize=(10,10))
for i in range(100):
    plt.subplot(10, 10, i + 1)
    img = sample['samples'][i]
    plt.imshow(img.cpu().numpy().transpose(1, 2, 0))
    plt.axis('off')
plt.tight_layout()
plt.savefig(os.path.join(args.sample_dir, 'whole_sample.png'), dpi=300)
plt.close()

for i in range(5):
    img = sample['progressive_samples'][i]
    img = make_grid(img, nrow=args.prog_sample_freq)
    plt.imshow(img.cpu().numpy().transpose(1, 2, 0))
    plt.tight_layout()
    plt.imsave(os.path.join(args.sample_dir, f'prog_sample_{i}.png'), img.cpu().numpy().transpose(1, 2, 0),dpi=300)






In [ ]:
%pylab inline
import matplotlib.image as mpimg

fig=plt.figure(figsize=(4,4), dpi= 200)
plt.axis('off')
img = mpimg.imread('samples/whole_sample.png')
imgplot = plt.imshow(img)
plt.show()

for i in range(5):
    img = mpimg.imread('samples/prog_sample_'+str(i)+'.png')
    plt.axis('off')
    imgplot = plt.imshow(img)
    plt.show()